In [1]:
import yfinance as yf
import datetime
from typing import List, Tuple

In [2]:

class StockPriceFetcher:
    """
    Fetches historical stock prices using Yahoo Finance.
    """

    def __init__(self, symbol: str):
        self.symbol = symbol

    def get_price_change(self, date: str, window: int = 1) -> Tuple[float, float]:
        """
        Fetch stock price on a given date and compare it with the next trading day to calculate price change.
        
        Parameters:
        - date (str): Date in 'YYYY-MM-DD' format
        - window (int): Number of days to check the change (default: 1 day)
        
        Returns:
        - Tuple of initial price, price change percentage
        """
        # Convert to datetime
        event_date = datetime.datetime.strptime(date, "%Y-%m-%d")
        
        # Fetch historical prices (1 week before and 1 week after event)
        start_date = (event_date - datetime.timedelta(days=7)).strftime("%Y-%m-%d")
        end_date = (event_date + datetime.timedelta(days=window + 7)).strftime("%Y-%m-%d")
        
        stock_data = yf.download(self.symbol, start=start_date, end=end_date)
        
        # Find the closest date after the event date with available price
        event_price = stock_data['Close'][event_date.strftime("%Y-%m-%d"):].head(1)
        future_price = stock_data['Close'][(event_date + datetime.timedelta(days=window)).strftime("%Y-%m-%d"):].head(1)

        if not event_price.empty and not future_price.empty:
            price_change = ((future_price.values[0] - event_price.values[0]) / event_price.values[0]) * 100
            return event_price.values[0], price_change
        else:
            return None, None

# Example usage
symbol = "RELIANCE.NS"
date = "2023-10-10"  # Date when the event happened

fetcher = StockPriceFetcher(symbol)
initial_price, change_percentage = fetcher.get_price_change(date)

if initial_price and change_percentage is not None:
    print(f"Initial Price: {initial_price}")
    print(f"Price Change: {change_percentage:.2f}%")
else:
    print("Could not fetch price data for the given date.")



[*********************100%***********************]  1 of 1 completed

Initial Price: 2308.39990234375
Price Change: 1.59%


In [6]:
!pip install spacy 


ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.

[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import spacy

class EventExtractor:
    """
    Extracts structured events (Actor, Action, Object) from news text using spaCy.
    """

    """
    Extracts structured events (Actor, Action, Object) from news text using spaCy.
    """

    def __init__(self):
        # Load spaCy model with dependency parsing and NER
        self.nlp = spacy.load("en_core_web_sm")

    def extract_events(self, text: str):
        """
        Extracts (Actor, Action, Object) tuples from a given news sentence.

        Parameters:
        - text (str): The news text.

        Returns:
        - List[Tuple[str, str, str]]: List of extracted events as (Actor, Action, Object)
        """
        doc = self.nlp(text)
        events = []

        for token in doc:
            # Identify the Actor (subject) and Action (verb)
            if token.dep_ in ("nsubj", "nsubjpass") and token.head.pos_ == "VERB":
                subject = token.text
                action = token.head.lemma_  # Use the base form of the verb
                obj = None

                # Look for a direct object or a prepositional object
                for child in token.head.children:
                    if child.dep_ == "dobj":  # Direct object takes priority
                        obj = child.text
                        # Check if there are multiple words (e.g., "Future Retail")
                        if child.nbor().dep_ in ("compound", "amod"):
                            obj = f"{child.nbor().text} {obj}"
                        break  # Stop once direct object is found
                    elif child.dep_ == "prep" and obj is None:
                        # Check for prepositional phrase (e.g., "in a deal")
                        for prep_obj in child.children:
                            if prep_obj.dep_ == "pobj":
                                obj = prep_obj.text

                # Ensure that the extracted object is meaningful
                if subject and action and obj:
                    events.append((subject, action, obj))

        return events


# Example usage
text = "Reliance acquired Future Retail in a massive deal."
extractor = EventExtractor()
extracted_events = extractor.extract_events(text)

for event in extracted_events:
    print(f"Actor: {event[0]}, Action: {event[1]}, Object: {event[2]}")


Actor: Reliance, Action: acquire, Object: Retail


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import BertModel, BertTokenizer
from typing import List, Tuple

# Initialize BERT model and tokenizer for word embeddings
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")



class EventEmbeddingNN(nn.Module):
    """
    Neural Network to create event embeddings from structured events.
    """

    def __init__(self, embedding_dim: int = 768):
        super(EventEmbeddingNN, self).__init__()
        self.embedding_dim = embedding_dim
        self.tensor1 = nn.Parameter(torch.Tensor(embedding_dim, embedding_dim))
        self.tensor2 = nn.Parameter(torch.Tensor(embedding_dim, embedding_dim))
        self.tensor3 = nn.Parameter(torch.Tensor(embedding_dim, embedding_dim))
        self.linear = nn.Linear(embedding_dim * 2, embedding_dim)
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.tensor1)
        nn.init.xavier_uniform_(self.tensor2)
        nn.init.xavier_uniform_(self.tensor3)

    def forward(self, actor_embed, action_embed, object_embed):
        r1 = torch.matmul(actor_embed, self.tensor1) * action_embed
        r2 = torch.matmul(action_embed, self.tensor2) * object_embed
        r3 = torch.matmul(r1 + r2, self.tensor3)
        event_embedding = torch.tanh(r3)
        return event_embedding

def get_bert_embeddings(text: str) -> torch.Tensor:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=10)
    outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

def create_event_embedding(actor: str, action: str, obj: str) -> torch.Tensor:
    actor_embedding = get_bert_embeddings(actor)
    action_embedding = get_bert_embeddings(action)
    object_embedding = get_bert_embeddings(obj)
    model = EventEmbeddingNN()
    event_embedding = model(actor_embedding, action_embedding, object_embedding)
    return event_embedding

# Example usage
event_extractor = EventExtractor()
news_text = "Reliance acquires Future Retail in a massive deal."
events = event_extractor.extract_events(news_text)

for event in events:
    actor, action, obj = event
    event_embedding = create_event_embedding(actor, action, obj)
    print(f"Event: {actor} {action} {obj}")
    print(f"Event Embedding: {event_embedding}")



Event: Reliance acquire Retail
Event Embedding: tensor([[-2.6472e-01, -3.1939e-02,  1.3612e-01, -1.4870e-01,  1.1417e-01,
         -2.4648e-01, -1.2438e-02, -6.7706e-02,  2.0345e-01, -5.8373e-02,
         -8.3977e-02,  2.2473e-01,  8.1046e-02, -9.5086e-03, -5.3481e-02,
          1.0855e-01,  1.8052e-01,  2.7730e-01, -4.2701e-02, -7.1269e-02,
          1.6633e-01,  9.2759e-02, -2.6806e-01, -1.3962e-01, -2.4098e-03,
          1.1058e-01,  3.3477e-02,  5.0030e-02,  6.6385e-02,  1.6248e-01,
          3.0844e-01, -5.7202e-02, -1.6269e-01,  3.9590e-02,  1.2307e-01,
          1.9050e-01,  3.2989e-02,  1.0000e-01, -2.1193e-01, -8.6090e-02,
         -1.0319e-01, -1.5863e-02, -7.1535e-02, -2.1982e-01,  2.4551e-01,
          1.8111e-01, -1.0920e-01,  3.2452e-02,  5.3849e-02, -1.4618e-01,
         -1.6637e-01,  3.8592e-02,  1.4367e-01, -2.9222e-01, -1.5433e-01,
         -1.0358e-02,  1.3742e-01,  1.9011e-01,  9.0533e-02, -1.2577e-01,
          1.2615e-01,  2.0031e-02,  1.9259e-01, -1.0177e-01,  1.